<a href="https://colab.research.google.com/github/KhaledRobot/ML/blob/main/Court_agent_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `law` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `law`


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

class LocalLLM:
    def __init__(self, model_name="microsoft/phi-2", device='cpu'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
        self.device = device

    def generate(self, prompt, max_new_tokens=256, temperature=0.7):
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        outputs = self.model.generate(
            **inputs, max_new_tokens=max_new_tokens, temperature=temperature, do_sample=True
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:


# Two–Lawyer Agents (Defense & Prosecution)


from __future__ import annotations
import os
from typing import List, Dict
from huggingface_hub import InferenceClient

class LawyerAgent:


    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = "microsoft/Phi-3-mini-4k-instruct"):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []      # list of {"role": ..., "content": ...}
        self.client = InferenceClient(
            model,
            token=os.getenv("HF_API_TOKEN")           # make sure this env‑var is set
        )

    # ---- helper for HF prompt formatting ----------
    def _format_prompt(self, user_msg: str) -> str:
        """
        Formats a full prompt that includes
        * system prompt
        * prior turns
        * new user message
        """
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        # HF text-generation endpoints expect a single string.

        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"
        return prompt

    # ---- produce a reply --------------------------
    def respond(self, user_msg: str, **gen_kwargs) -> str:
        prompt = self._format_prompt(user_msg)
        completion = self.client.text_generation(
            prompt,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True,
            stream=False,
            **gen_kwargs
        )
        answer = completion.strip()
        # keep chat memory
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer


# System prompts

DEFENSE_SYSTEM = """
You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
"""

PROSECUTION_SYSTEM = """
You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good‑faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""

PLAINTIFF_SYSTEM = """
You are **Garcia**, *Plaintiff* in this case. Your role:
• Clearly state what the defendant did wrong
• Explain how their actions harmed you
• Specify what compensation/remedy you seek
Style:
• Factual and concise (1-2 sentences per point)
• Avoid emotional language; focus on damages
"""

WITNESS_SYSTEM = """
You are an **eyewitness**. Rules:
• ONLY state what you directly observed
• Never speculate or guess
• If unsure, say "I don't recall"
Example responses:
• "I saw the defendant enter the server room at 11PM"
• "I don't know what they did inside"
"""

DEFENDANT_SYSTEM = """
You are the defendant. Your job:
- Deny or explain the accusations against you
- Stick to facts you personally know
- Be respectful but firm
- Admit uncertainty where needed
"""

JUDGE_SYSTEM = """
You are the **Judge**. Your role:
- Summarize briefly what each side argued
- Decide guilt or innocence based on arguments
- Explain your reasoning simply
- Be neutral, fair, and concise
"""

# the two agents

defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)

# New agents
plaintiff = LawyerAgent("Plaintiff", PLAINTIFF_SYSTEM)
witness = LawyerAgent("Witness", WITNESS_SYSTEM)
defendant = LawyerAgent("Defendant", DEFENDANT_SYSTEM)
judge = LawyerAgent("Judge", JUDGE_SYSTEM)

# Example dialogue  |  State v. Doe

print("==== Opening statements ====\n")

case_background = (
    "The State alleges that John Doe stole proprietary algorithms from his former employer "
    "and used them at a competitor. The charge is felony theft of trade secrets. "
    "No physical evidence shows direct copying, but server logs indicate large downloads "
    "two days before Doe resigned."
)

# Prosecutor goes first
p_open = prosecution.respond(
    f"Opening statement to the Court. Background: {case_background}"
)
print("PROSECUTOR:", p_open, "\n")

# Defense responds
d_open = defense.respond(
    "Opening statement to the Court responding to the prosecution."
)
print("DEFENSE   :", d_open, "\n")

# Prosecutor rebuttal
p_rebut = prosecution.respond(
    "Brief rebuttal to the defense's opening."
)
print("PROSECUTOR:", p_rebut, "\n")

# ====== NEW ADDITIONS ======
print("\n==== Plaintiff's Testimony ====")  # NEW
plaint = plaintiff.respond(
    f"Describe your damages to the Court. Background: {case_background}"
)
print("PLAINTIFF:", plaint, "\n")  # NEW

print("==== Witness Examination ====")  # NEW
# Prosecutor questions witness
p_question = prosecution.respond(
    "Ask the witness one specific question about the server logs."
)
print("PROSECUTOR to Witness:", p_question)  # NEW
w_answer = witness.respond(p_question)  # NEW
print("WITNESS:", w_answer, "\n")  # NEW

# Defense cross-examination  # NEW
d_question = defense.respond(
    "Ask the witness one follow-up question to challenge their testimony."
)
print("DEFENSE to Witness:", d_question)  # NEW
w_answer2 = witness.respond(d_question)  # NEW
print("WITNESS:", w_answer2, "\n")  # NEW

# Final arguments referencing new testimony  # NEW
print("==== Closing Arguments ====")  # NEW
p_close = prosecution.respond(  # NEW
    f"Closing statement incorporating witness testimony: '{w_answer}' and plaintiff statement: '{plaint}'"
)
print("PROSECUTOR:", p_close, "\n")  # NEW

d_close = defense.respond(  # NEW
    f"Final argument addressing witness testimony: '{w_answer2}'"
)
print("DEFENSE:", d_close)  # NEW


==== Opening statements ====

PROSECUTOR: "Ladies and Gentlemen of the jury, today, you will be presented with clear and convincing evidence that John Doe, my client's former employer, a company dedicated to innovation and hard work, has been the victim of a serious infringement of their rights. This crime is not just about stolen intellectual property; it's about the fundamental principles of fairness and respect for another's hard work that underpin our society.


John Doe, a skilled software engineer, played a critical role in developing proprietary algorithms essential to the success of my client's business. These algorithms are not only the result of countless hours of dedication but also a form of intellectual property protected under the law. On the day before John Doe left his job, there were large downloads from the company's servers, indicating he may have accessed confidential information.


My client, a pioneer in its field, has invested heavily in research and development.

In [ ]:
# ... (keep all your existing imports and LawyerAgent class)

class CourtAgent(LawyerAgent):
    def __init__(self, name: str, role: str, case_text: str):
        system_prompt = f"""
        **ROLE**: {role} in this case
        **CASE TEXT**: "{case_text[:300]}..."
        **RULES**:
        1. Respond ONLY about this specific case
        2. Never mention other cases/laws
        3. Keep responses under 100 words
        4. If unsure, say "Objection - beyond case scope"
        """
        super().__init__(name, system_prompt)
        self.case_text = case_text

# Load cases
import pandas as pd
from typing import List, Dict

# Load the CSV file without headers
try:
    # Assuming the first column contains the case text
    cases_df = pd.read_csv("data.csv", header=None, names=['full_case_text'])
    print(f"Successfully loaded {len(cases_df)} cases")
except Exception as e:
    print(f"Error loading data: {e}")
    cases_df = pd.DataFrame(columns=['full_case_text'])  # empty dataframe as fallback

# Process cases
results = []
MAX_CASES = min(10, len(cases_df))  # Handle case if fewer than 10 cases exist

for i, row in cases_df.head(MAX_CASES).iterrows():
    try:
        case_text = row['full_case_text']

        # Initialize agents (using your existing LawyerAgent class)
        prosecutor = LawyerAgent("Prosecutor",
                               f"You are prosecuting this case. Case details: {case_text[:300]}...")
        defense = LawyerAgent("Defense",
                             f"You are defending this case. Case details: {case_text[:300]}...")

        # Generate arguments
        prosecution_stmt = prosecutor.respond("Present your opening arguments to the court")
        defense_stmt = defense.respond("Present your response to the prosecution's arguments")

        results.append({
            'case_id': i,
            'case_preview': str(case_text)[:100] + "...",  # str() in case it's not string
            'prosecution': prosecution_stmt,
            'defense': defense_stmt
        })
    except Exception as e:
        print(f"Error processing case {i}: {e}")

# Preview results
print("\nSample outputs:")
for r in results[:2]:
    print(f"\nCase {r['case_id']}: {r['case_preview']}")
    print(f"Prosecution: {r['prosecution'][:100]}...")
    print(f"Defense: {r['defense'][:100]}...")


Successfully loaded 501 cases
Error processing case 8: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/microsoft/Phi-3-mini-4k-instruct (Request ID: Root=1-680293a8-006bd8644c29f25049440aff;7013c92a-7ebe-43fb-a04c-8c1951876f79)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.
Error processing case 9: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/microsoft/Phi-3-mini-4k-instruct (Request ID: Root=1-680293a8-589ba77b675cf2b2533a9087;a4c7aa0b-0e66-455c-b5d3-8d4f44435cef)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

Sample outputs:

Case 0: text...
Prosecution: Ladies and Gentlemen of the jury, before you today we have an issue of substantial importance that t...
Defense: I'm sorry, but I cannot generate a response without know

In [ ]:
# Convert to DataFrame and save
results_df = pd.DataFrame(results)
results_df.to_csv('court_results.csv', index=False)
files.download('court_results.csv')  # Auto-download in Colab

print(f"\n🎉 Saved {len(results_df)} case results to court_results.csv")
print("Columns:", results_df.columns.tolist())